## Imports

In [ ]:
import pathlib
import pickle

import matplotlib.pyplot as plt
import tqdm
import torch
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import DataLoader

from super_resolution.src.sen2venus_dataset import (
    create_train_validation_test_split,
    default_patch_transform,
)
from super_resolution.src.visualization import plot_gallery

## Constants

In [ ]:
DATA_DIR = pathlib.Path("C:/Users/Mitch/stat3007_data")
SITES_DIR = DATA_DIR / "sites"
PREPROCESSING_DIR = DATA_DIR / "preprocessing"
RESULTS_DIR = DATA_DIR / "results"

## Basic data loading

In [ ]:
train_patches, val_patches, test_patches = create_train_validation_test_split(
    str(SITES_DIR) + "\\", sites={"K34-AMAZ"}
)
print(
    f"Num train {len(train_patches)}\n"
    f"Num validation {len(val_patches)}\n"
    f"Num test {len(test_patches)}"
)

In [ ]:
train_loader = DataLoader(train_patches, batch_size=2)

In [ ]:
for i, (low_res, high_res) in enumerate(train_loader):
    print(
        f"batch {i}\n"
        f"low resolution batch shape {low_res.shape}\n"
        f"high resolution batch shape {high_res.shape}\n"
    )

    if i == 2:
        break

In [ ]:
import random

index = random.randint(0, len(low_res) - 1)

low_res_example = low_res[index]
high_res_example = high_res[index]

plot_gallery(

    [low_res_example.permute(1, 2, 0), high_res_example.permute(1, 2, 0)],

    xscale=5,

    yscale=5,
)

## Data loading with transforms

We often want to apply some transforms calculated from our training set and apply these
to both the training set and test sets before running data through our model. These custom transforms
are useful for training stability and normalization. This gives us another component of the model we
can experiment with if we choose.

In [ ]:
# ~2min
# Compute mean and standard deviation for each channel (averaged over samples).
train_loader = DataLoader(train_patches, batch_size=1000)

mean = 0.0
std = 0.0
for i, (low_res, _) in enumerate(train_loader):
    mean += torch.sum(torch.mean(low_res, (2, 3)), 0)
    std += torch.sum(torch.std(low_res, (2, 3)), 0)
mean /= len(train_patches)
std /= len(train_patches)
print(f"Channel means {mean}\n" f"Channel standard deviations {std}\n")

Computing the mean and std can be slow. Here is some code for saving and loading the data.

In [ ]:
save_file = PREPROCESSING_DIR / "K34-AMAZ_mean_std.pkl"

In [ ]:
# with open(save_file, "wb") as file:
#     pickle.dump((mean, std), file)

In [ ]:
# with open(save_file, "rb") as file:
#     mean, std = pickle.load(file)

In [ ]:
def custom_transform(
    low_res_patch: torch.Tensor, high_res_patch: torch.Tensor
) -> tuple[torch.Tensor, torch.Tensor]:
    augmentations = transforms.Compose(
        [
            transforms.Normalize(mean, std),
        ]
    )

    # Basic transforms such as removing 4th channel and scaling surface reflectants
    low_res_patch, high_res_patch = default_patch_transform(
        low_res_patch, high_res_patch
    )

    # torchvision transforms expects shape (CxHxW) so permute accordingly
    low_res_patch = low_res_patch.permute(0, 2, 1)
    low_res_augmented = augmentations(low_res_patch).permute(0, 2, 1)

    return low_res_augmented, high_res_patch

In [ ]:
train_patches.set_transform(custom_transform)

Images will now be loaded with the above custom transform applied.

## Example Model

In [ ]:
class SimpleSR(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3, 3, 3, 1, 65)

    def forward(self, x):
        return self.conv(x)

In [ ]:
model = SimpleSR()
print(model)

## Train Loop Example

In [ ]:
NUM_EPOCHS = 2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
should_pin_memory = {"cuda": True, "cpu": False}

train_loader = DataLoader(
    train_patches,
    shuffle=True,  # Always set to true for training
    batch_size=128,  # Always try to set as powers of 2
    drop_last=True,  # Ensures batch size is always the one given (Drops last if its is smaller)
    pin_memory=should_pin_memory[device.type],  # Faster push to GPU
    num_workers=2,  # Load data in parallel but costs more memory
)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
# ~21min for 10 epochs
model = model.to(device)
model.train()

losses = []
epoch_losses = []
for epoch in range(NUM_EPOCHS):
    progress_bar = tqdm.tqdm(train_loader, total=len(train_loader), ncols=100)
    epoch_loss = 0.0
    for low_res_batch, high_res_batch in progress_bar:
        # Push to GPU
        low_res_batch = low_res_batch.to(device)
        high_res_batch = high_res_batch.to(device)

        optimizer.zero_grad()

        # Loss and update
        out = model(low_res_batch)
        loss = criterion(out, high_res_batch)
        loss.backward()
        optimizer.step()

        # Collect data
        epoch_loss += loss.item()
        losses.append(loss.item())
        progress_bar.set_postfix(epoch=epoch, batch_loss=loss.item())

    epoch_loss /= len(train_loader)
    epoch_losses.append(epoch_loss)
    print(f"Epoch: {epoch} / loss: {epoch_loss:.3f}")

In [ ]:
experiment = {
    "epoch_losses": epoch_losses,
    "batch_losses": losses,
    "model_state": model.state_dict(),
    "optimizer_state": optimizer.state_dict(),
}
save_file = RESULTS_DIR / "basic_sr_results.pkl"

In [ ]:
# with open(save_file, "wb") as file:
#     pickle.dump(experiment, file)

In [ ]:
# with open(save_file, "rb") as file:
#     experiment = pickle.load(file)

In [ ]:
plt.plot(losses)
plt.title("Losses for each minibatch")
plt.xlabel("Minibatch")
plt.ylabel("MSE")
plt.show()

In [ ]:
plt.plot(epoch_losses)
plt.title("Losses for each epoch")
plt.xlabel("Epoch")
plt.ylabel("Mean MSE")
plt.show()

## Inference Example

In [ ]:
with torch.no_grad():
    out = model(low_res_example.unsqueeze(0)).squeeze(0)
out = (out - out.min()) / (out.max() - out.min())
plot_gallery(
    [
        low_res_example.permute(1, 2, 0),
        high_res_example.permute(1, 2, 0),
        out.permute(1, 2, 0),
    ],
    xscale=5,
    yscale=5,
)